# SOLID design principles

Design principles described by uncle Bob in 2000.

- S: Single responsibility
- O: Open/Closed
- L: Liskov substitution
- I: Interface segregation
- D: Dependency Inversion

The following is a step by step guide on how to apply these principles, an example from ArjanCodes video material (source1)[add_link_to_source]

Example of applying principles for a sale system having:
- items
- quantities
- prices 
- payment status
and functions for:
- adding items
- computing order items
- paying order

## Step 0 (without SOLID principles):

In [4]:
class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total
    
    def pay(self, payment_type, security_code):
        if payment_type == 'debit':
            print ('Processing debit payment type...')
            print (f'Verifying security code: {security_code}...')
            self.status = 'paid'
        elif payment_type == 'credit':
            print ('Processing credit payment type...')
            print (f'Verifying security code: {security_code}...')
            self.status = 'paid'
        else:
            raise Exception(f'Unknown payment type: {payment_type}')

In [5]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print('Order total:', order.total_price())
order.pay('debit', '01263874')

Order total: 180
Processing debit payment type...
Verifying security code: 01263874...


# Principle 1: Single Responsibility

Classes and methods should have a single responsibility, be responsible for only 1 thing.

### Why?

This ensures:
- simplicity in code.
- makes things 'Easier to Change' (ETC value, [TPP]), easier modification/expansion 
- easier testing. 

### How is this not applied in V0?

Class Order is responsible for: 
1) Handling order (adding items & Calculating total price)
2) Managing payment

Additionally, method 'pay' manages both 'credit' and 'debit' payments.

### Step to take:
- Extract payment process to it's own class.
- Separate each method to handle one kind of payment
- Since the new class will need to change the status of the payment from 'open' to 'paid',
the new class can either: 
    - take the order as an argument and change it's status attribute or
    - define a set_status method in order class. 
     

In [20]:
#V1:
class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total
    
class PaymentProcessor:
        
    def pay_debit(self, order, security_code):
        print ('Processing debit payment type...')
        print (f'Verifying security code: {security_code}...')
        order.status = 'paid'
        
    def pay_credit(self, order, security_code):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {security_code}...')
        order.status = 'paid'

In [21]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print('Order total:', order.total_price())
processor = PaymentProcessor()
processor.pay_debit(order=order, security_code='01263874')


Order total: 180
Processing debit payment type...
Verifying security code: 01263874...


- Applied Single Responsibility by introducing a separate class to deal with payments, and two separate methods dealing with each payment type.

- However, we introduced coupling, tight coupling between classes Order and PaymentProcessor:
    - Direct Manipulation of Order's Status: The PaymentProcessor directly changes the status attribute of the Order instance (order.status = 'paid'). This direct manipulation signifies tight coupling because the PaymentProcessor needs intimate knowledge of the internal structure of the Order class. A change in the Order class's status attribute (like renaming it) would require changes in the PaymentProcessor class as well.

    - Lack of Abstraction: The PaymentProcessor interacts directly with concrete attributes of the Order class. There is no abstraction layer (like an interface or use of getter/setter methods) that separates the PaymentProcessor's understanding of an Order from the actual implementation of the Order. This makes the code less flexible and more fragile to changes.

 

# Principle 2: Open/Closed

Code should be **Open** for extension for adding new functionalities,
but **closed** for modification, existing code should not be modified in order to add new functionalities

### How existing code violates this principle?

If we want to add a new payment method, we have to modify the PaymentProcessor class. This violates the Open/Closed principle.

### Approach to fix that:

Create a structure of Classes and Subclasses, so that we can add a new subclass if we want to add a new payment type.

(My thoughts: Maybe adding classes is not always necessary, but instead of adding classes for each method, we could only add methods. 
Probably for large projects, this creates technical debt quickly though.)

**Abstract Class**: is a blue-print for any other class that inherits from this. It allows you to create a set of **abstract methods** that must be created 
within any subclasses inheriting from the abstract class. Used when:
    - designing large functional units, or 
    - want to provide a common interface for different implementations of a component. 

**Abstract Method**: is a method that is declared but has no implementation. 

### Abstract Class & Method in python:

In python abc module is used to define abstract classes & methods:
i.e.

    from abc import ABC, abstractmethod
    #ABC used for abstract  class
    #abstractmethod used for abstract method
    
    class PaymentProcessor(ABC):
        @abstractmethod
        def pay(self, order, security_code):
            pass


In [25]:
#V2:
from abc import ABC, abstractmethod

class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total
    
class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order, security_code):
        pass
    
class DebitPaymentProcessor(PaymentProcessor):
        
    def pay(self, order, security_code):
        print ('Processing debit payment type...')
        print (f'Verifying security code: {security_code}...')
        order.status = 'paid'

class CreditPaymentProcessor(PaymentProcessor):
        
    def pay(self, order, security_code):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {security_code}...')
        order.status = 'paid'
        
#adding another payment type:
class PaypalPaymentProcessor(PaymentProcessor):
        
    def pay(self, order, security_code):
        print ('Processing PayPal payment type...')
        print (f'Verifying security code: {security_code}...')
        order.status = 'paid'

In [26]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print('Order total:', order.total_price())
#processor = CreditPaymentProcessor()
processor = PaypalPaymentProcessor()
processor.pay(order=order, security_code='01263874')



Order total: 180
Processing PayPal payment type...
Verifying security code: 01263874...


# Liskov Substitution Principle (LSP)

'A subtype should behave like a supertype as far as you can tell by using the supertype methods' 

For classes: Subclasses that inherit from another parent class, should have implementations of all abstract methods defined in the parent class.

### How existing code violates this principle?

The new PaypalPaymentProcessor class inherits from PaymentProcessor, **BUT** abuses it in a way. 
Why? PaypalPaymentProcessor needs an email and not a security code. 'security_code' parameter is defined in the PaymentProcessor, and thus the
parent class expects to have a security code passed in the 'security_code' variable and not an email.  

### Approaches to fix that if violated:

- 1) Add an initializer in each subclass for 'security_code' or 'e-mail' respectively. (My note: note sure if this does not violate LSP...)
- 2) Use PaymentProcessor as a grandparent class, define two parent classes: PaymentProcessorWithSecurityCode and PaymentProcessorWithEmail inheriting 
from grandparent and defining new abstract method security_code and e-mail respectively, and inherit from these two parent classes to the final payment 
classes.

In [35]:
#V3:
from abc import ABC, abstractmethod

class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total
    
class PaymentProcessor(ABC):
    @abstractmethod
    def pay(self, order):
        pass
    
class DebitPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        
    def pay(self, order):
        print ('Processing debit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'

class CreditPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        
    def pay(self, order):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'
        
#adding another payment type:
class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self,e_mail):
        self.e_mail = e_mail
        
    def pay(self, order):
        print ('Processing PayPal payment type...')
        print (f'Verifying security code: {self.e_mail}...')
        order.status = 'paid'

In [39]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print('Order total:', order.total_price())
processor = PaypalPaymentProcessor(e_mail='hi@this.is')
processor.pay(order=order)

print ('_'*15)

print('Order total:', order.total_price())
processor = CreditPaymentProcessor(security_code='13258403')
processor.pay(order=order)


Order total: 720
Processing PayPal payment type...
Verifying security code: hi@this.is...
_______________
Order total: 720
Processing credit payment type...
Verifying security code: 13258403...


# Interface Segregation Principle (ISP)

It is better to have several specifiv interfaces instead of one general interface. 

The reason for this principle, according to [CA] is that if you have a general interface that covers a lot of operations
(lets say op1, op2, op3), and a user uses only one operation (user1 -> op1) then if you change source code of op2, this
will force user 1 to recompile. This will happen in java, but not in python.

In dynamically typed languages like python (and not statistically typed languages) howver, this doesn't happen. ....

However, it is better not to depend on something that carries baggage that you don't need. 

    # Note: to better illustrate the principle, imagine that the 
    # following abstract method exists in the PaymentProcessor(ABC) 
    # abstract class prior modifying it to follow ISP:
    @abstractmethod
    def auth_sms(self, code):
      pass
    # and the method is defined in the DebitPaymentProcessor:
    def auth_sms(self, code):
      print (f'Verifying SMS code {code}')
      self.verified = True
    (Imagine code prior applying ISP was as the following, V3':)


In [45]:
#V3'

from abc import ABC, abstractmethod

class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total
    
class PaymentProcessor(ABC):
    
    @abstractmethod
    def auth_sms(self, code):
        pass
    
    @abstractmethod
    def pay(self, order):
        pass
    
class DebitPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        self.verified = False  
                
      
    def auth_sms(self, code):
      print (f'Verifying SMS code {code}')
      self.verified = True
        
    def pay(self, order):
        if not self.verified:
            raise Exception('Not authorized')
        print ('Processing debit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'

class CreditPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        
    def auth_sms(self, code):
        raise Exception("Credut card payments don't support sms authentication.")
    
    def pay(self, order):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'
        
#adding another payment type:
class PaypalPaymentProcessor(PaymentProcessor):
    
    def __init__(self,e_mail):
        self.e_mail = e_mail
        self.verified = False  
        
    def auth_sms(self, code):
      print (f'Verifying SMS code {code}')
      self.verified = True

    def pay(self, order):
        if not self.verified:
            raise Exception('Not authorized')
        print ('Processing PayPal payment type...')
        print (f'Verifying security code: {self.e_mail}...')
        order.status = 'paid'

In [48]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print ('_'*15)

print('Order total:', order.total_price())
processor = CreditPaymentProcessor(security_code='13258403')
processor.pay(order=order)

print ('_'*15)

print('Order total:', order.total_price())
processor = PaypalPaymentProcessor(e_mail='hi@this.is')
processor.auth_sms('0485')
processor.pay(order=order)

_______________
Order total: 540
Processing credit payment type...
Verifying security code: 13258403...
_______________
Order total: 540
Verifying SMS code 0485
Processing PayPal payment type...
Verifying security code: hi@this.is...


### How does V3' violate the ISP?

PaymentProcessor class has an abstract function (auth_sms) defined that is not used by all the subclasses of the PaymentProcessor. 

### How to fix it?

1) Similarly with what was described before, use more subclasses:
add another subclass that defines an abstract 
method auth_sms, to be inherited by the class/classes that use the auth_sms.
(class PaymentProcessor_SMS(PaymentProcessor):
    @abstractmethod
    def aut_sms(self, code):
        pass
)

2) Use Composition:
Define a separate independent class that handles auth_sms (This is shown in V4), 
separating differnet kinds of behaviour.

In [63]:
#V4

from abc import ABC, abstractmethod

class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total

class SMSAuth:
    
    def __init__(self):
        self.authorized = False

    def verify_code(self, code): 
        # for simplicity assuming that all codes are verified
        print(f'Verifying code {code}')
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized


class PaymentProcessor(ABC):
       
    @abstractmethod
    def pay(self, order):
        pass


class DebitPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code, authorizer: SMSAuth):
        self.authorizer = authorizer
        self.security_code = security_code
        
    def pay(self, order):
        if not self.authorizer.is_authorized():
            raise Exception('Not authorized')
        print ('Processing debit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'

class CreditPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        
    
    def pay(self, order):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'
        
#adding another payment type:
class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self,email_address, authorizer: SMSAuth):
        self.authorizer = authorizer
        self.email_address = email_address
        
    def pay(self, order):
        if not self.authorizer.is_authorized():
            raise Exception('Not authorized')
        print ('Processing PayPal payment type...')
        print (f'Verifying e-mail: {self.email_address}...')
        order.status = 'paid'

In [64]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print ('_'*15)

print('Order total:', order.total_price())
processor = CreditPaymentProcessor(security_code='13258403')
processor.pay(order=order)

print ('_'*15)

print('Order total:', order.total_price())
sms_authorizer = SMSAuth()
processor = PaypalPaymentProcessor(email_address='hi@this.is', authorizer = sms_authorizer)
sms_authorizer.verify_code(46546)
processor.pay(order=order)

_______________
Order total: 180
Processing credit payment type...
Verifying security code: 13258403...
_______________
Order total: 180
Verifying code 46546
Processing PayPal payment type...
Verifying e-mail: hi@this.is...


# Dependency Inversion Principle (DIP)

"The Dependency Inversion Principle (DIP) tells us that the most flexible systems are
those in which source code dependencies refer only to abstractions, not concetions."  [CA]

Classes should depend on abstractions instead of concrete subclasses. 

Note from [CA]: Treading this idea as a rule is unrealistic, some times software systems
must depend on concrete facilities. However, we want to avoid having a system depending 
on volatile concrete elements.

### How does V4 violate the DIP?

Classes "DebitPaymentProcessor" & "PaypalPaymentProcessor" depend on "SMSAuth" object, which
comes from a well defined class.

### So what?

In case that we want to add more authenticators available, this will be a problem. 
Remembering "Easier to Change" (ETC) value, by using DIP, our code will be 
easier to change.

### How do we do that?

-   1. Create an abstract class.
-   2. Make "SMSAuth" a subclass of the created class.
-   3. Update expected type of authorizer from SMSAuth to the abstract class. 
    *Using the type hints in the code will not affect the code itself, however it
    will affect code quality, and make things easier to read and modify -> ETC

In [72]:
#V5

from abc import ABC, abstractmethod

class Order:
    items = []
    quantities = []
    prices = []
    status = 'open'
    
    def add_item(self, name, quantity, price):
        self.items.append(name)
        self.quantities.append(quantity)
        self.prices.append(price)
    
    def total_price(self):
        total = 0
        for i in range (len(self.prices)):
            total += self.quantities[i] * self.prices[i]
        return total

class Authorizer(ABC):
    
    @abstractmethod
    def is_authorized(self) -> bool:
        pass

class SMSAuth(Authorizer):
    
    def __init__(self):
        self.authorized = False

    def verify_code(self, code): 
        # for simplicity assuming that all codes are verified
        print(f'Verifying code {code}')
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized
    
class NotARobot(Authorizer):
    # Example of adding another type of authorizor
    
    def __init__(self):
        self.authorized = False

    def not_a_robot(self): 
        # for simplicity assuming that all codes are verified
        print(f'Are you a robot? Naaaa')
        self.authorized = True

    def is_authorized(self) -> bool:
        return self.authorized


class PaymentProcessor(ABC):
       
    @abstractmethod
    def pay(self, order):
        pass


class DebitPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code, authorizer: Authorizer):
        self.authorizer = authorizer
        self.security_code = security_code
        
    def pay(self, order):
        if not self.authorizer.is_authorized():
            raise Exception('Not authorized')
        print ('Processing debit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'

class CreditPaymentProcessor(PaymentProcessor):

    def __init__(self,security_code):
        self.security_code = security_code
        
    
    def pay(self, order):
        print ('Processing credit payment type...')
        print (f'Verifying security code: {self.security_code}...')
        order.status = 'paid'
        
#adding another payment type:
class PaypalPaymentProcessor(PaymentProcessor):

    def __init__(self,email_address, authorizer: Authorizer):
        self.authorizer = authorizer
        self.email_address = email_address
        
    def pay(self, order):
        if not self.authorizer.is_authorized():
            raise Exception('Not authorized')
        print ('Processing PayPal payment type...')
        print (f'Verifying e-mail: {self.email_address}...')
        order.status = 'paid'

In [73]:
#Placing order & payment process:
order = Order()
order.add_item('Keyboard', 1, 20)
order.add_item('SSD', 1, 150)
order.add_item('USB cable', 2, 5)

print ('_'*15)

print('Order total:', order.total_price())
processor = CreditPaymentProcessor(security_code='13258403')
processor.pay(order=order)

print ('_'*15)

print('Order total:', order.total_price())
sms_authorizer = SMSAuth()
processor = PaypalPaymentProcessor(email_address='hi@this.is', authorizer = sms_authorizer)
sms_authorizer.verify_code(46546)
processor.pay(order=order)

print ('_'*15)

print('Using another authorizer:')
print('Order total:', order.total_price())
authorizer = NotARobot()
processor = PaypalPaymentProcessor(email_address='hi@this.is', authorizer = authorizer)
authorizer.not_a_robot()
processor.pay(order=order)

_______________
Order total: 180
Processing credit payment type...
Verifying security code: 13258403...
_______________
Order total: 180
Verifying code 46546
Processing PayPal payment type...
Verifying e-mail: hi@this.is...
_______________
Using another authorizer:
Order total: 180
Are you a robot? Naaaa
Processing PayPal payment type...
Verifying e-mail: hi@this.is...


## Sources: 

1) https://www.youtube.com/watch?v=pTB30aXS77U  (+ Arjan's youtube content)

2) [TPP] 'The Pragmatic Programmer your journey to mastery' by David Thomas and Andrew Hunt

3) [CC] 'Clean Code A Handbook of Agile Software Craftmaship' by Robert C. Martin

4) [CA] 'Clean Architecture A Craftman's Guide to Software Structure and Design' by Robert C. Martin

5) Liskov explaining her pinciple: https://www.youtube.com/watch?v=-Z-17h3jG0A

## OOP sources:

1) Abstract Classes: https://www.geeksforgeeks.org/abstract-classes-in-python/